# Setup


In [ ]:
! pip install datasets einops sentencepiece tokenizers
! pip install git+https://github.com/huggingface/transformers

# Download


In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

device = "cuda"
# model = AutoModelForCausalLM.from_pretrained("Open-Orca/Mistral-7B-OpenOrca").to(device)
tokenizer = AutoTokenizer.from_pretrained("Open-Orca/Mistral-7B-OpenOrca")

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
tokenizer.encode("bot", add_special_tokens=False), tokenizer.encode("user", add_special_tokens=False) 

([12435], [2188])

In [6]:
tokenizer.encode("<s>", add_special_tokens=False), tokenizer.encode("</s>", add_special_tokens=False) 

([1], [2])

In [10]:
chat = [
    {
        "role": "system",
        "content": "You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!",
    },
    {
        "role": "user",
        "content": "How are you?",
    },
    {
        "role": "assistant",
        "content": "I am doing well!",
    },
    {
        "role": "user",
        "content": "Please tell me about how mistral winds have attracted super-orcas.",
    },
]
chat_tokenizer_template = tokenizer.apply_chat_template(
    chat,
    # tokenize=True,
    tokenize=False,
    add_generation_prompt=True,
)
print(chat_tokenizer_template)

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


<|im_start|>system
You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!<|im_end|>
<|im_start|>user
How are you?<|im_end|>
<|im_start|>assistant
I am doing well!<|im_end|>
<|im_start|>user
Please tell me about how mistral winds have attracted super-orcas.<|im_end|>



# Run


In [14]:
inputs = tokenizer.apply_chat_template(
    chat,
    tokenize=True,
    # tokenize=False,
    # add_generation_prompt=True,
    return_tensors='pt'
).to(device)
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=256,
    use_cache=True,
    do_sample=True,
    temperature=0.2,
    top_p=0.95
)
text = tokenizer.batch_decode(outputs)[0]
print(text)


Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


<|im_start|> system
You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!<|im_end|><|im_start|> user
How are you?<|im_end|><|im_start|> assistant
I am doing well!<|im_end|><|im_start|> user
Please tell me about how mistral winds have attracted super-orcas.<|im_end|> 

Step 1: Understand the question
The user is asking about the relationship between mistral winds and super-orcas.

Step 2: Research the topic
Mistral winds are strong, cold, and dry winds that blow from the northeast in the Mediterranean region. Super-orcas are a term used to describe large, powerful orcas (killer whales) that are known for their impressive size and strength.

Step 3: Analyze the relationship between the two
There is no direct relationship between mistral winds and super-orcas. Mistral winds are a weather phenomenon, while super-orcas are a type of marine animal. However, it is possible that the presence of mistr

In [5]:
sys_prompt = "A chat."
# prompt = "Tell me a joke."
# prompt = "Напиши алгоритм как погладить котика"
prompt = "Сколько у человека пальцев. Ответь с научной точки зрения."

prefix = "<|im_start|>"
suffix = "<|im_end|>\n"
sys_format = prefix + "system\n" + sys_prompt + suffix
user_format = prefix + "user\n" + prompt + suffix
assistant_format = prefix + "assistant\n"
input_text = sys_format + user_format + assistant_format

generation_config = GenerationConfig(
    max_length=256,
    temperature=1.1,
    top_p=0.95,
    repetition_penalty=1.0,
    do_sample=True,
    use_cache=True,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    transformers_version="4.34.0.dev0",
)

inputs = tokenizer(input_text, return_tensors="pt", return_attention_mask=True).to(
    device
)
outputs = model.generate(**inputs, generation_config=generation_config)

text = tokenizer.batch_decode(outputs)[0]
print(text)

<s><|im_start|> system
A chat.<|im_end|><|im_start|> user
Сколько у человека пальцев. Ответь с научной точки зрения.<|im_end|><|im_start|> assistant
 У взрослого человека на каждой руке по 5 пальцев. На ногах у взрослого человека также по 5 пальцев на каждом пне. Однако у детей количество пальцев может быть немного больше на начальных стадиях развития.<|im_end|>


# Debug


In [ ]:
print(tokenizer.eos_token_id)
print(model.config.eos_token_id)
print(tokenizer.pad_token_id)
print(model.config.pad_token_id)

2
2
None
None


In [ ]:
text = tokenizer.decode([1, 2, 32000, 32001])
print(text)

<s></s><|im_end|><|im_start|>


In [ ]:
tokenizer.convert_tokens_to_ids("<|im_end|>")

32000